# Merge Osquery Data from different systems
After running Osquery on different systems use this Jupyter Notebook to add the data into one big CSV file.

## Import needed modules

In [ ]:
# Standard modules to use and manipulate dataframes
import numpy as np
import pandas as pd

## Variables and Functions

In [ ]:
location = %pwd
csvlocation = location + "\\CSV\\"

In [ ]:
def read_csv_data(filename):
    df_r = pd.read_csv(csvlocation + filename)
    return(df_r)

## Read-in Osquery Data

In [ ]:
#Read in data
data_orig = read_csv_data('data_total_dup_LNX.csv')
data_add = read_csv_data('data_total_dup_WIN.csv')

#NOFILTER
print(data_orig.shape)
print(data_add.shape)

## Create the `hash` based on same `anon` and filter them out
The `anon` data is used per DataFrame, if the DataFrame already has the `hash` columns skip it, otherwise the data will not be correct anymore. As you can get collisions with same `anon` but different `hash` because off the addition of other DataFrame to that data. 

In [ ]:
if not 'hash' in data_orig:
    #groupby and create hash of connections
    dict_orig_hash = pd.util.hash_pandas_object(data_orig.groupby('anon')['Table.Column'].apply(', '.join), index=False).to_dict()
    #add hash to df
    data_orig['hash'] = data_orig['anon'].map(dict_orig_hash)
    print(data_orig.shape)

if not 'hash' in data_add:
    #groupby and create hash of connections
    dict_add_hash = pd.util.hash_pandas_object(data_add.groupby('anon')['Table.Column'].apply(', '.join), index=False).to_dict()
    #add hash to df
    data_add['hash'] = data_add['anon'].map(dict_add_hash)
    print(data_add.shape)

In [ ]:
#find out hashes that are in `data_add` and not in `data_orig`, if you change the order you will get more looking at the other DF but that is not a correct view.
data_add_diff = data_add[~data_add.hash.isin(data_orig.hash)].sort_values(by='hash')
data_add_diff.shape

In [ ]:
data_add_diff.nunique()

## Add the diff data to the source data
Drop `anon` column as it is not unique anymore.

In [ ]:
FORMAT = ['Table','Table.Column','Column','hash'] #,'anon'
data_add_diff = data_add_diff[FORMAT]
data_orig = data_orig[FORMAT]
print(data_orig.shape)
print(data_add_diff.shape)
data_total = pd.concat([data_add_diff,data_orig])
print(data_total.shape)
data_total.to_csv(csvlocation + 'data_total_dup_LNX+WIN.csv', index=False)